# Example 3: Synthetic beam collections: stats & merge

**Goal.** Generate a collection of *artificial* beams for:
- computing **statistics** for each run and for the whole collection,
- **merging** all runs into a single standardized beam and compute stats again.


In [ ]:
__author__ = ['Rafael Celestre']
__contact__ = 'rafael.celestre@synchrotron-soleil.fr'
__license__ = 'CECILL-2.1'
__copyright__ = 'Synchrotron SOLEIL, Saint Aubin, France'
__created__ = '2025.11.10'
__changed__ = '2025.11.10'

import numpy as np
import pandas as pd

import barc4beams as b4b

## 1) Synthetic beam generator

In [ ]:
N_RUNS   = 50          # number of synthetic beams
N_RAYS   = 1000        # rays per beam
SEED     = 42          # base RNG seed
E0       = 720         # central photon energy (eV)
VAR_PCT  = 0.10        # ~±10% variability (1σ) for parameters
INT_MEAN = 1.0/np.sqrt(2.0)  # target mean intensity ~0.707
DEAD_FRAC  = 0.10      # ~10% lost rays

# Base spatial/angle widths (1σ) at source plane
SIG_X0   = 17.31e-6      # m
SIG_Y0   = 13.40e-6      # m
SIG_DX0  = 23.67e-6      # rad
SIG_DY0  = 23.02e-6      # rad

# Target focal distances (sign = where the waist lies relative to z0)
F_X0     = -0.1        # m (focus upstream)
F_Y0     = +0.1        # m (focus downstream)

In [ ]:

def _mixture_normal(rng, n, sigma, skew_weight=1, skew_shift_sigma=1):
    '''Return a 1D sample with slight skew/kurtosis using a 2-Gaussian mixture.'''
    k = int(skew_weight * n)                 
    base = rng.normal(0.0, sigma, n - k)
    skew = rng.normal(skew_shift_sigma*sigma, sigma, k)
    return np.concatenate([base, skew])

def synth_beam_df(rng, n_rays, params, dead_frac=0.10):
    '''
    Build one synthetic standardized beam DataFrame.

    params dict keys:
        sig_x, sig_y, sig_dx, sig_dy, fx, fy, energy_eV, int_mean
    '''
    sig_x  = params["sig_x"];  sig_y  = params["sig_y"]
    sig_dx = params["sig_dx"]; sig_dy = params["sig_dy"]
    fx     = params["fx"];     fy     = params["fy"]
    e_eV   = params["energy_eV"]
    int_mu = params["int_mean"]

    X = _mixture_normal(rng, n_rays, sig_x, skew_weight=0.10, skew_shift_sigma=0.8)
    Y = _mixture_normal(rng, n_rays, sig_y, skew_weight=0.10, skew_shift_sigma=0.8)

    eps_dx = rng.normal(0.0, 0.20 * sig_dx, n_rays)
    eps_dy = rng.normal(0.0, 0.20 * sig_dy, n_rays)

    dX = (-1.0 / fx) * X + eps_dx
    dY = (-1.0 / fy) * Y + eps_dy

    I = np.clip(rng.normal(int_mu, 0.10 * int_mu, n_rays), 0.0, 1.0)

    lost_mask = rng.random(n_rays) < dead_frac
    I[lost_mask] = 0.0

    I_s = 0.6 * I
    I_p = 0.4 * I

    lam = b4b.misc.energy_wavelength(e_eV, "eV")

    df = pd.DataFrame({
        "energy":     np.full(n_rays, e_eV, dtype=float),
        "X":          X.astype(float),
        "Y":          Y.astype(float),
        "dX":         dX.astype(float),
        "dY":         dY.astype(float),
        "wavelength": np.full(n_rays, lam, dtype=float),
        "intensity":  I.astype(float),
        "intensity_s-pol": I_s.astype(float),
        "intensity_p-pol": I_p.astype(float),
        "lost_ray_flag":   lost_mask.astype(np.uint8),
    })
    return df


## 2) Build a collection of beams

In [ ]:
rng = np.random.default_rng(SEED)

def vary(val, rel_sigma=0.10, keep_sign=False):
    '''Return val * (1 + N(0, rel_sigma)), preserving sign if requested.'''
    scale = 1.0 + rng.normal(0.0, rel_sigma)
    out = val * scale
    if keep_sign and np.sign(out) != np.sign(val):
        out = abs(out) * np.sign(val)
    return out

def vary_uniform(val, rel_sigma=0.10, keep_sign=False):
    '''Return val * (1 + U[-rel_sigma, +rel_sigma]), preserving sign if requested.'''
    scale = 1.0 + rng.uniform(-rel_sigma, rel_sigma)
    out = val * scale
    if keep_sign and np.sign(out) != np.sign(val):
        out = abs(out) * np.sign(val)
    return out

runs = []
for i in range(N_RUNS):
    params = {
        "sig_x":     vary(SIG_X0),
        "sig_y":     vary(SIG_Y0),
        "sig_dx":    vary(SIG_DX0),
        "sig_dy":    vary(SIG_DY0),
        "fx":        vary(F_X0, keep_sign=True),
        "fy":        vary(F_Y0, keep_sign=True),
        "energy_eV": vary_uniform(E0),
        "int_mean":  vary_uniform(1.0/np.sqrt(2.0)),
    }
    df = synth_beam_df(rng, N_RAYS, params, dead_frac=DEAD_FRAC)
    runs.append(df)

len(runs), [r.shape for r in runs[:2]]


## 3) Statistics across runs

In [ ]:

stats_multi = b4b.get_statistics(runs, verbose=True)
stats_multi

## 4) Merge and compute stats

In [ ]:
merged = b4b.merge_standard_beams(runs)
beam_merged = b4b.Beam(merged)
stats_merged = beam_merged.stats(verbose=True)

## 5) Quick visuals

In [ ]:
plot = beam_merged.plot_beam(mode="hist2d", aspect_ratio=True, color=3, path=None, bins=100)

In [ ]:
plot = beam_merged.plot_divergence(mode="hist2d", aspect_ratio=True, color=3, path=None, bins=100)

In [ ]:
plot = beam_merged.plot_phase_space(direction="both", mode="hist2d", aspect_ratio=False, color=3, dpi=100, path=None, plot=True)

In [ ]:
plot = beam_merged.plot_caustic(which="both", n_points=101, xy_range=[-100,100], start=-0.2, finish=0.2, color=3)